In [1]:
#Dependencies
import pandas as pd
import requests
import json
import datetime
from config import api_key


In [2]:
states_abbrev_file_path = "Resources/states_abbrev.csv"

states_abbrev_df = pd.read_csv(states_abbrev_file_path, encoding='ISO-8859-1')
states_abbrev_df

,Airport,City,Abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB
...,...,...,...,...,...
367,Yampa Valley Airport,Hayden,CO,Colorado,HDN
368,Yeager Airport,Charleston,WV,West Virginia,CRW
369,Yellowstone Regional Airport,Cody,WY,Wyoming,COD
370,Youngstown-Warren Regional Airport,Youngstown / Warren,OH,Ohio,YNG


In [3]:
top_25_airports = [
    'ATL',
    'LAX',
    'ORD',
    'DFW',
    'DEN',
    'JFK',
    'SFO',
    'SEA',
    'LAS',
    'MCO',
    'EWR',
    'CLT',
    'PHX',
    'IAH',
    'MIA',
    'BOS',
    'MSP',
    'FLL',
    'DTW',
    'PHL',
    'LGA',
    'BWI',
    'SLC',
    'SAN',
    'IAD']


In [4]:
states_abbrev_df = states_abbrev_df.rename(columns = {'Abbrev': 'state_abbrev'})
states_abbrev_df.head()

,Airport,City,state_abbrev,State,IATA
0,Aberdeen Regional Airport,Aberdeen,SD,South Dakota,ABR
1,Abilene Regional Airport,Abilene,TX,Texas,ABI
2,Abraham Lincoln Capital Airport,Springfield,IL,Illinois,SPI
3,Akron-Canton Regional Airport,Akron / Canton,OH,Ohio,CAK
4,Albany International Airport,Albany,NY,New York,ALB


In [5]:
# API Call for covid data 
covid_base_url = "https://api.covidtracking.com/v1/states/daily.json"
response = requests.get(covid_base_url).json()

#print(json.dumps(response, indent=4))

# pull the data from a source df to a target df by creating a new df

covid_df = pd.DataFrame(response, columns=['date', 'state', 'positive', 'death', 'hospitalized', 'totalTestResults'])
covid_df.head()



,date,state,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [6]:
covid_df = covid_df.rename(columns = {'state': 'state_abbrev'})
covid_df.head()

,date,state_abbrev,positive,death,hospitalized,totalTestResults
0,20201021,AK,12644.0,68.0,NaN,547140.0
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0
3,20201021,AS,0.0,0.0,NaN,1616.0
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0


In [7]:
covid_df['Date'] = ''
covid_df.head()

        

,date,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,20201021,AK,12644.0,68.0,NaN,547140.0,
1,20201021,AL,174528.0,2805.0,19330.0,1273523.0,
2,20201021,AR,101596.0,1751.0,6526.0,1242752.0,
3,20201021,AS,0.0,0.0,NaN,1616.0,
4,20201021,AZ,233912.0,5854.0,20820.0,1657389.0,


In [8]:

for x, y in covid_df.iterrows():
    
    org_date = str(y.get(key='date'))
    
    year_s = org_date[0:4]
    month_s = org_date[4:6]
    day_s = org_date[6:8]
    formatted_date = f"{year_s}-{month_s}-{day_s}"
    
    covid_df.at[x, 'Date'] = str(formatted_date)
  
    
        
covid_df = covid_df.drop('date', axis=1)
covid_df.head(20)


,state_abbrev,positive,death,hospitalized,totalTestResults,Date
0,AK,12644.0,68.0,NaN,547140.0,2020-10-21
1,AL,174528.0,2805.0,19330.0,1273523.0,2020-10-21
2,AR,101596.0,1751.0,6526.0,1242752.0,2020-10-21
3,AS,0.0,0.0,NaN,1616.0,2020-10-21
4,AZ,233912.0,5854.0,20820.0,1657389.0,2020-10-21
5,CA,877784.0,17027.0,NaN,17293139.0,2020-10-21
6,CO,87582.0,2060.0,8332.0,1724038.0,2020-10-21
7,CT,64871.0,4567.0,12043.0,2070126.0,2020-10-21
8,DC,16498.0,642.0,NaN,476443.0,2020-10-21
9,DE,23375.0,670.0,NaN,331480.0,2020-10-21


In [9]:


no_of_days = 5

curr_date = pd.to_datetime('today').date()
curr_date =  pd.to_datetime(curr_date)


#START DATE 
start_date = str(curr_date)
start_date = start_date.split(' ')[0]


#END DATE
end_date = datetime.datetime.today() - datetime.timedelta(days=no_of_days)
end_date = str(end_date)
end_date = end_date.split(' ')[0]

print(start_date)
print(end_date)


2020-10-21
2020-10-16


In [10]:
#Generate a list that will hold all the flight dates we would like to query

curr_date =datetime.datetime.today() - datetime.timedelta(days=no_of_days)
query_dates = []

for i in range(1,no_of_days + 1):
    

    Next_date = curr_date + datetime.timedelta(days=1)

    str_date =  pd.to_datetime(Next_date)
    str_date = str(str_date)
    str_date = str_date.split(' ')[0]
    
    curr_date = Next_date
    query_dates.append(str_date)
    
query_dates
   

['2020-10-17', '2020-10-18', '2020-10-19', '2020-10-20', '2020-10-21']

In [14]:
# API Call for flights data 

url = f"http://api.aviationstack.com/v1/flights?access_key={api_key}&"
#query_date = ['2020-10-01','2020-10-02','2020-10-03' ]
query_iata = ['ATL','LAX','ORD']
#query_iata = top_25_airports
value_list = []

for q_date in query_dates:
    for q_iata in query_iata:
        flights_base_url = f"{url}flight_date={q_date}&arr_iata={q_iata}&flight_status=landed&limit=1"
        flights_response = requests.get(flights_base_url).json()
        landed_flights = flights_response['pagination']['total']
        arr_IATA = flights_response['data'][0]['arrival']['iata']
    
       
        value_list.append({'Date': q_date, 'IATA':arr_IATA, 'Total Landed Flights': landed_flights})
        
flights_df = pd.DataFrame(value_list)   
flights_df


,Date,IATA,Total Landed Flights
0,2020-10-17,ATL,63
1,2020-10-17,LAX,82
2,2020-10-17,ORD,67
3,2020-10-18,ATL,2454
4,2020-10-18,LAX,1184
5,2020-10-18,ORD,1996
6,2020-10-19,ATL,2152
7,2020-10-19,LAX,1106
8,2020-10-19,ORD,1861
9,2020-10-20,ATL,1998


In [12]:
#Merge states data with flights data

merged_df = states_abbrev_df.merge(flights_df, on='IATA')
merged_df.head(50)

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-17,67
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507
4,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1094
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-17,63
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152
8,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998
9,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1801


In [13]:
#Merge Flight data with COVID data

merged_df = merged_df.merge(covid_df, how='inner', left_on=["Date", "state_abbrev"], right_on=["Date","state_abbrev"])
merged_df

,Airport,City,state_abbrev,State,IATA,Date,Total Landed Flights,positive,death,hospitalized,totalTestResults
0,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-17,67,343390.0,9452.0,NaN,6699844.0
1,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-18,1996,347635.0,9474.0,NaN,6779140.0
2,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-19,1861,350748.0,9496.0,NaN,6827824.0
3,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-20,1507,354462.0,9537.0,NaN,6886901.0
4,Chicago O'Hare International Airport,Chicago,IL,Illinois,ORD,2020-10-21,1094,358804.0,9605.0,NaN,6953692.0
5,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-17,63,339384.0,7607.0,30350.0,3280668.0
6,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-18,2454,340558.0,7638.0,30376.0,3298458.0
7,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-19,2152,341310.0,7657.0,30388.0,3314279.0
8,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-20,1998,342438.0,7674.0,30541.0,3327836.0
9,HartsfieldJackson Atlanta International Airport,Atlanta,GA,Georgia,ATL,2020-10-21,1801,342438.0,7674.0,30541.0,3327836.0
